In [1]:
# Jupyter, Python imports
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

#  python 3interactive figures in a live IPython notebook session
%matplotlib nbagg

In [95]:
# studentproject18ws imports
import os
import logging
from studenproject18ws.hdf.reader import Reader
from studenproject18ws.hdf.recipes import Recipes

# filename = 'banddos_4x4.hdf'
# filename = 'banddos.hdf'
filename = 'banddos_Co.hdf'

filepath = ['..', 'data', 'input', filename]
filepath = os.path.join(*filepath)

data = None
extractor = Reader(filepath=filepath)
with extractor as h5file:
    data = extractor.read(recipe=Recipes.Bands)
    #
    # Note:
    # Inside the with statement (context manager),
    # all data attributes that are type h5py Dataset are available (in-file access)
    # When the statement is left,the HDF5 file gets closed and the datasets are closed.
    #
    # Use data outside the with-statement (in-memory access: all HDF5 datasets converted to numpy ndarrays):
    data.move_datasets_to_memory()
    


INFO:root:Loaded datasets.
DEBUG:root:Transforming datasets: ------------------------------------------------
DEBUG:root:i = 0, dataset = 'atoms_position', h5path = '/atoms/positions', transform = '[['TransformBands.coordinates', <LatticeType.Bravais: 1>]]':
DEBUG:root:	check dependencies:
DEBUG:root:		for transform_function: 'TransformBands.coordinates'
DEBUG:root:	is dependent on untransformed datasets ['reciprocalCell', 'bravaisMatrix'], try next dataset first
DEBUG:root:i = 1, dataset = 'atoms_group', h5path = '/atoms/equivAtomsGroup', transform = '['Transform.id']':
DEBUG:root:	check dependencies:
DEBUG:root:		for transform_function: 'Transform.id'
DEBUG:root:	not dependent or all dependencies satisfied, transform:
DEBUG:root:		transform_function: 'Transform.id', transform_args: '['atoms_group', <HDF5 dataset "equivAtomsGroup": shape (16,), type "<i4">]'
DEBUG:root:	transformed dataset 'atoms_group'.
DEBUG:root:i = 2, dataset = 'bandUnfolding', h5path = '/general', transform = '[[

DEBUG:root:	check dependencies:
DEBUG:root:		for transform_function: 'Transform.slicer'
DEBUG:root:	not dependent or all dependencies satisfied, transform:
DEBUG:root:args_additional before: ['[0]']
DEBUG:root:args_additional after: ['[0]']
DEBUG:root:		transform_function: 'Transform.slicer', transform_args: '['unused_ksym', <HDF5 dataset "ksym": shape (1, 416, 49), type "<i4">, '[0]']'
DEBUG:root:	transformed dataset 'unused_ksym'.
DEBUG:root:i = 16, dataset = 'unused_numFoundEigenvalues', h5path = '/eigenvalues/numFoundEigenvals', transform = '['Transform.id']':
DEBUG:root:	check dependencies:
DEBUG:root:		for transform_function: 'Transform.id'
DEBUG:root:	not dependent or all dependencies satisfied, transform:
DEBUG:root:		transform_function: 'Transform.id', transform_args: '['unused_numFoundEigenvalues', <HDF5 dataset "numFoundEigenvals": shape (1, 416), type "<i4">]'
DEBUG:root:	transformed dataset 'unused_numFoundEigenvalues'.
DEBUG:root:i = 0, dataset = 'atoms_position', h5path 

In [37]:
# more imports if necessary
from collections import namedtuple

In [81]:
# data (currently) has a function that returns a plt configured to .show() bandstrure plt.
# we want to reuse that code here now. So do a little magic: retrieve the function definiton

def retrieve_function_def(function):
    """Returns the function definition of a func in memory and pastes into new notebook cell.
    """
    import inspect
    get_ipython().set_next_input("".join(inspect.getsourcelines(function)[0]))

retrieve_function_def(data.new_plotfunction_weights)

In [ ]:
    def new_plotfunction_weights(self, bands, characters, groups, spin):
        """

        :param bands:
        :param characters:
        :param groups:
        :param spin:
        :return:
        """

        weight_k_n = self.combined_weight(characters, groups, spin)

        fig = plt.figure()
        ax1 = fig.add_subplot(111)

        for n in bands:
            weight_k_n[n]
            ax1.scatter(self.k_distances, self.E_i(n, spin), marker='o', c='b', s=2 * weight_k_n[n], lw=0)

        plt.xticks(self.k_special_points, self.k_special_point_labels)

        """
        for i in range(len(special_points)):
            index = special_points[i]
            plt.vlines(k_dist[index-1], -0.2, 0.4)
        """
        plt.xlim(0, max(self.k_distances))
        return plt


In [98]:
# okay, copied the relevant code from data.new_plotfunction_weights:
# defining the plt outside of the function so we can link it with widgets.

fig, ax = plt.subplots(1, figsize=(10,6))
plt.suptitle(f"BandStructure of {filename}")

# define widgets function


# bands = atom_group_keys = e.g. for banddos.hdf: dict_keys[(1,2,3,4,5)]
#                           Hm... should better convert to tuple back in reader?
def_groups = data.atom_group_keys
select_groups = widgets.SelectMultiple(options=def_groups, 
                                   value=tuple(def_groups),
                                  description='Atom Groups',
                                  disabled=False)


def_characters = ['s', 'p', 'd', 'f']
# Characters = namedtuple('Characters', ['s', 'p', 'd', 'f'])
# characters = Characters(0,1,2,3)
select_characters = widgets.SelectMultiple(options=def_characters,
                                          value=tuple(def_characters),
                                          description='Band Character',
                                          disabled=False)

# number of bands can be large, so use a rangeslider instead of selectionslider
def_bands = [band for band in range(data.eigenvalues.shape[2])]
select_bands = widgets.IntRangeSlider(value=[def_bands[0]+1,def_bands[-1]+1], 
                                      min=def_bands[0]+1, max=def_bands[-1]+1, step=1,
                                     description='Bands',
                                     disabled=False, continuous_update=False,
                                     orientation='horizontal', readout=True,
                                     readout_format='d')

@interact(bands=select_bands, characters=select_characters, groups=select_groups)
def update_plot(bands, characters, groups):
    
    ax.clear()
    
    # convert arguments to the expected format
    bands_conved = range(bands[0]-1,bands[1]-1)
    groups_conved = [el-1 for el in groups]
    characters_conved = [def_characters.index(el) for el in characters]
    
    # TODO: define spin also as parameter
    spin = 0
    weight_k_n = data.combined_weight(characters=characters_conved, groups=groups_conved, spin=spin)
    
    for n in bands_conved:
        weight_k_n[n]
        ax.scatter(data.k_distances, data.E_i(n, spin), marker='o', c='b', s=2 * weight_k_n[n], lw=0)
    
    plt.xticks(data.k_special_points, data.k_special_point_labels)
    
    plt.xlim(0, max(data.k_distances))
    plt.show()
    



<IPython.core.display.Javascript object>

interactive(children=(IntRangeSlider(value=(1, 49), continuous_update=False, description='Bands', max=49, min=…